In [ ]:
#Importing Libraries
import numpy  as np
import pandas as pd
from sklearn.feature_extraction.text  import TfidfVectorizer

In [ ]:
# K-Means
from sklearn import cluster
from sklearn.cluster import KMeans


# Visualization and Analysis
import matplotlib.pyplot  as plt
import seaborn as sns
from sklearn.metrics import silhouette_samples, silhouette_score
from wordcloud import WordCloud


In [40]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import spacy
import multiprocessing
from gensim.models.phrases import Phrases, Phraser
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from gensim.models import Word2Vec


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [49]:

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [29]:
#Importing dataset
from sklearn.datasets import fetch_20newsgroups

In [30]:
#Reading the 20Newsgroup dataset 
df = fetch_20newsgroups(subset='all',remove=('headers', 'footers', 'quotes'),shuffle=True, random_state=102)

In [31]:
#No of labels
labels = df.target
n=len(np.unique(labels))
n

20

In [32]:
corpus = df.data

In [38]:
corpus[0:3]

['You definetly are in need of a shrink, loser!',
 "Software that comes together with the VideoBlaster\nis designed to work together with the SoundBlaster\n(from the same manufacturer).\n\nSince I do not own a SoundBlaster: is there a possibility\nto use the PC Speaker driver to play audio files for\nthe VideoBlaster (.AVI = audio video interleave files) ?\n\nI think what I should have is a device driver for\nthe Media Player that controls the PC Speaker Driver\ninstead of the SoundBlaster card (something like MCISPKR.DRV).\n\nHas anybody heard of such a driver?\nOr am I on the wrong track?\n\nAny information on this appreciated!\n\n(Please send e-mail, since I don't watch this group regularly.)\n",
 'I have an old IBM PC-XT motherboard which has TWO banks\nof dip switches (eight switches per bank).  I need to \nknow which switch is required to install a hard disk.\n\nDoes anyone have any archived documentation that would\nhelp me?  \n                                            \nThank

In [43]:
corpus[0].replace('[^\w\s]','')        


'You definetly are in need of a shrink, loser!'

In [47]:
# Function for preprocessing
def pre_processing(docs):
  stop = stopwords.words('english')

  for doc in docs:
      raw_text = doc.lower()
      doc_punc = doc.replace('[^\w\s]','')        
      # tokenization
      tokens_text = word_tokenize(raw_text)
      # remove stopwords
      stopped_tokens_text = [i for i in tokens_text if not i in stop]
      # remoce digis and one-charcter word
      doc = [token for token in stopped_tokens_text if not token.isnumeric()]
      doc = [token for token in stopped_tokens_text if len(token) > 1]
      # you could always add some new preprocessing here
      yield doc


In [50]:
# Preprocess all the documents in the corpus
Vocab_v1 = list(pre_processing(corpus))

In [51]:
print(len(Vocab_v1))
print(Vocab_v1[0])


18846
['definetly', 'need', 'shrink', 'loser']


In [53]:
#Training Word2Vec model

# Initialize and train the model. 
print("Training the Word2Vec model...");
w2v_model = Word2Vec(Vocab_v1, size=100, min_count = 20, window = 5, sample = 1e-3)
# calling init_sims to make the model more memory efficient
w2v_model.init_sims(replace=True);
# Save the model
model_name = "model_word2vec_20newsGroup";
w2v_model.save(model_name);

Training the Word2Vec model...


In [57]:
word_vectors = w2v_model.wv.syn0;


<ipython-input-57-885e4218ff0c>:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  word_vectors = w2v_model.wv.syn0;


In [58]:
print(word_vectors[0].shape)


(100,)


In [59]:
# cluster the text using KMeans
num_clusters = 20
kmeans_clustering = KMeans(n_clusters = num_clusters)
clusters = kmeans_clustering.fit_predict(word_vectors)
centers = kmeans_clustering.cluster_centers_

In [60]:
# Calculate Silhoutte Score
#
score = silhouette_score(word_vectors, kmeans_clustering.labels_, metric='euclidean')
#
# Print the score
#
print('Silhouetter Score: %.3f' % score)

Silhouetter Score: 0.121
